For this analysis, we use the cells that were used for the typing, as we are sure that they have a proper spatial STA (needed for LSC measure).

In [25]:
# rootF = 'C:/Users/Sarah/Documents/Myopia/Data analysis/Blur experiments/'
rootF = 'D:/Data/Paper defocus/Defocus detection data/'

In [8]:
import os as os
import sys
import numpy as np
import csv
import pickle as pickle
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import matplotlib.patches as pcs
from skimage.transform import rescale, resize, downscale_local_mean
import seaborn as sns
from matplotlib.patches import Ellipse

# Import functions
from shared.functions import plot_scale_bar, load_image_as_on_MEA
from shared.LSC_functions import local_spatial_contrast
from shared.save_load import load_obj, save_obj

In [ ]:
exp = 20211208

# Experiment conditions
from shared.parameters_stimulation2 import *
pixel_size_um = 3.5
pixel_per_check = 12
nb_checks = 60

['1.40', '2.00']


# Load data

In [26]:
path_to_original_images = rootF + '/Eye model simulations/original images/'
imF = rootF + '/Eye model simulations/convolved images/'
path_to_PSFs =  rootF + '/Eye model simulations/mouse PSFs/'

##### STAs

In [16]:
cell_data = load_obj(rootF + '/MEA/exp' + str(exp) + '/cells_characterization_data_' + str(exp))

data_clusters = list(cell_data.keys())

##### Select STA

In [ ]:
results_path = rootF + '/MEA/exp' + str(exp) + '/'
    
cluster_selection = pd.read_excel(results_path + str(exp) + '_cluster_selection.xlsx', header=0)

cluster_names = cluster_selection['Cluster']
defined_STA = cluster_selection['Comments Sarah']
STA_start = cluster_selection['Spatial STA']
STA_end = cluster_selection['Sp STA END']

skip_cells = [] # very low firing rate during chirp

# Select "YES" cells with a spatial STA
selected_cells = []
which_STA = []

for i in range(len(cluster_names)):
    if cluster_names[i] not in skip_cells and defined_STA[i] != 'no STA':
        if STA_start[i] == 0:
            print (cluster_names[i], 'end')
            which_STA.append('end')
            selected_cells.append(cluster_names[i])
        else:
            print (cluster_names[i], 'start')
            which_STA.append('start')
            selected_cells.append(cluster_names[i])
            
print (len(selected_cells), 'good cells!')

5 start
10 start
20 start
22 start
41 start
45 start
55 start
56 start
62 start
63 start
66 start
73 end
78 start
83 start
86 start
99 start
100 end
102 start
112 start
119 start
125 start
126 start
130 start
136 start
138 start
144 start
153 start
158 end
162 start
168 start
195 start
210 start
220 start
238 start
241 start
247 start
260 start
263 end
270 start
271 start
273 start
276 start
280 start
283 start
284 start
287 start
292 start
296 end
300 start
316 start
318 start
329 start
333 start
335 start
356 start
359 start
360 end
361 start
368 start
370 start
380 start
382 start
387 start
389 start
401 start
406 start
407 start
411 end
415 start
419 start
420 start
421 start
422 start
424 start
430 end
435 start
438 start
439 start
441 start
447 start
449 start
450 start
452 start
457 start
460 end
463 start
464 start
466 start
469 start
470 start
471 start
472 end
478 end
480 start
482 start
486 start
493 start
496 start
497 start
498 start
506 start
511 start
513 start
515 start

##### Cells with defined STA

In [21]:
exp_path = rootF + '/MEA/exp' + str(exp) + '/'

cells_clustering = load_obj(exp_path + str(exp) + '_selected_cells_clustering')
cells_clustering = np.array(cells_clustering)

print (len(cells_clustering), 'cells were typed')

113 cells were typed


# Local contrast and mean intensity in blurred images

In [22]:
pred_path = rootF + '/contrast analysis/local_contrast_2sigma_liu2022_exp' + str(exp) 

if exp == 20211129:
    skip_cells = [45, 80, 188, 192, 401, 408, 422, 525, 533, 558, 561, 614, 682]
elif exp == 20211207:
    skip_cells = [33, 52, 145, 302, 340, 406, 414, 515]
elif exp == 20211208:
    skip_cells = [99,100,276,407,424,452,472,498] # also 407, 452, 472 because too small ellipses
elif exp == 20221213:
    skip_cells = [55, 180, 232, 294, 339, 365, 563, 570, 665, 680]
elif exp == 20221214:
    skip_cells = [40, 190, 332, 957]
elif exp == 20230105:
    skip_cells = [15, 96, 177, 221, 274, 320, 326, 389, 498, 558, 571, 689, 700, 791, 832, 856, 923, 928, 1018]

# Load ellipse points within the 2sigma contour
ell_path = rootF + '/contrast analysis/ellipses_data_with_weights_2sigma_' + str(exp) 

ellipse_data_2s = load_obj(ell_path + '.pkl')

if os.path.isfile(pred_path + '.pkl'):
    print ('Loading predictions')
    local_contrasts = load_obj(pred_path + '.pkl')
    mean_intensities = load_obj(rootF + '/contrast analysis/mean_intensities_2sigma_liu2022_exp' + str(exp) + '.pkl')
else:
    print ('Calculating predictions...')

    local_contrasts = np.empty((len(cells_clustering),Nimgs,NpupDiams,Nsources,NdFs))
    mean_intensities = np.empty((len(cells_clustering),Nimgs,NpupDiams,Nsources,NdFs))

    for index in range(len(cells_clustering)):
        cell_nb = cells_clustering[index]
        print (cell_nb)
        if cell_nb in selected_cells and cell_nb not in skip_cells:
            ### Get ellipse points
            if exp in [20221213, 20221214, 20230105]:
                cell_nb = str(cell_nb)
            ellipsepoints = ellipse_data_2s[cell_nb]['ellipse_points']
            weights = ellipse_data_2s[cell_nb]['weights']
            ### Local contrast for each image
            for iImg in range(Nimgs):
                for iPup in range(NpupDiams):
                    for iEcc in range(Nsources):
                        for iDf in range(NdFs):
                            df = dFs[iDf]
                            # load blurred image
                            im = load_image_as_on_MEA(imF + 'image_{}pupDiam={}_dF='.format(Imgs[iImg], pupDiams[iPup]) \
                                            + df + 'Source_{}.png'.format(sources[iEcc]))
                            
                            ellipse_weights = weights[ellipsepoints[:,1], ellipsepoints[:,0]]
                            mean_int, lsc = local_spatial_contrast(im, ellipsepoints, ellipse_weights)
                            
                            mean_intensities[index,iImg,iPup,iEcc,iDf] = mean_int
                            local_contrasts[index,iImg,iPup,iEcc,iDf] = lsc

Loading predictions


In [ ]:
### Save predictions
save_path = rootF + '/contrast analysis/' 
                
save_obj(local_contrasts, save_path + '/local_contrast_2sigma_liu2022_exp' + str(exp))
save_obj(mean_intensities, save_path + '/mean_intensities_2sigma_liu2022_exp' + str(exp))

### With negative images

In [37]:
imF_neg = 'D:/Data/MEA data/inverted blurred images/Convolved images/'
pred_path = rootF + '/contrast analysis/neg_local_contrast_2sigma_liu2022_exp' + str(exp) 

if exp == 20211129:
    skip_cells = [45, 80, 188, 192, 401, 408, 422, 525, 533, 558, 561, 614, 682]
elif exp == 20211207:
    skip_cells = [33, 52, 145, 302, 340, 406, 414, 515]
elif exp == 20211208:
    skip_cells = [99,100,276,407,424,452,472,498] # also 407, 452, 472 because too small ellipses
elif exp == 20221213:
    skip_cells = [55, 180, 232, 294, 339, 365, 563, 570, 665, 680]
elif exp == 20221214:
    skip_cells = [40, 190, 332, 957]
elif exp == 20230105:
    skip_cells = [15, 96, 177, 221, 274, 320, 326, 389, 498, 558, 571, 689, 700, 791, 832, 856, 923, 928, 1018]

# Load ellipse points within the 2sigma contour
ell_path = rootF + '/contrast analysis/ellipses_data_with_weights_2sigma_' + str(exp) 

ellipse_data_2s = load_obj(ell_path + '.pkl')

if os.path.isfile(pred_path + '.pkl'):
    print ('Loading predictions')
    local_contrasts = load_obj(pred_path + '.pkl')
    mean_intensities = load_obj(rootF + '/contrast analysis/neg_mean_intensities_2sigma_liu2022_exp' + str(exp) + '.pkl')
else:
    print ('Calculating predictions...')

    local_contrasts = np.empty((len(cells_clustering),Nimgs,1,Nsources,NdFs))
    mean_intensities = np.empty((len(cells_clustering),Nimgs,1,Nsources,NdFs))

    for index in range(len(cells_clustering)):
        cell_nb = cells_clustering[index]
        print (cell_nb)
        if cell_nb in selected_cells and cell_nb not in skip_cells:
            ### Get ellipse points
            if exp in [20221213, 20221214, 20230105]:
                cell_nb = str(cell_nb)
            ellipsepoints = ellipse_data_2s[cell_nb]['ellipse_points']
            weights = ellipse_data_2s[cell_nb]['weights']
            ### Local contrast for each image
            for iImg in range(Nimgs):
                for iPup in [0]: #range(NpupDiams): # only 1.4mm
                    for iEcc in range(Nsources):
                        for iDf in range(NdFs):
                            df = dFs[iDf]
                            # load blurred image
                            im = load_image_as_on_MEA(imF_neg + 'image_{}pupDiam={}_dF='.format(Imgs[iImg], pupDiams[iPup]) \
                                            + df + 'Source_{}.png'.format(sources[iEcc]))
                               
                            ellipse_weights = weights[ellipsepoints[:,1], ellipsepoints[:,0]]
                            mean_int, lsc = local_spatial_contrast(im, ellipsepoints, ellipse_weights)
                            
                            mean_intensities[index,iImg,iPup,iEcc,iDf] = mean_int
                            local_contrasts[index,iImg,iPup,iEcc,iDf] = lsc

Calculating predictions...
5
10
20
22
41
45
55
56
62
63
66
73
78
83
86
99
100
102
112
119
125
126
130
136
138
144
153
158
162
168
195
210
220
238
241
247
260
263
270
271
273
276
280
283
284
287
292
296
300
316
318
329
333
335
356
359
360
361
368
370
380
382
387
389
401
406
407
411
415
419
420
421
422
424
430
435
438
439
441
447
449
450
452
457
460
463
464
466
469
470
471
472
478
480
482
486
493
496
497
498
506
511
513
515
529
532
544
546
576
580
592
605
613


In [38]:
### Save predictions
save_path = rootF + '/contrast analysis/' 
                
save_obj(local_contrasts, save_path + '/neg_local_contrast_2sigma_liu2022_exp' + str(exp))
save_obj(mean_intensities, save_path + '/neg_mean_intensities_2sigma_liu2022_exp' + str(exp))

# LSC and MI in sharp images

In [29]:
pred_path = rootF + '/contrast analysis/original_images_local_contrast_2sigma_liu2022_exp' + str(exp) 

if exp == 20211129:
    skip_cells = [45, 80, 188, 192, 401, 408, 422, 525, 533, 558, 561, 614, 682]
elif exp == 20211207:
    skip_cells = [33, 52, 145, 302, 340, 406, 414, 515]
elif exp == 20211208:
    skip_cells = [99,100,276,407,424,452,472,498] # also 407, 452, 472 because too small ellipses
elif exp == 20221213:
    skip_cells = [55, 180, 232, 294, 339, 365, 563, 570, 665, 680]
elif exp == 20221214:
    skip_cells = [40, 190, 332, 957]
elif exp == 20230105:
    skip_cells = [15, 96, 177, 221, 274, 320, 326, 389, 498, 558, 571, 689, 700, 791, 832, 856, 923, 928, 1018]

# Load ellipse points within the 2sigma contour
ell_path = rootF + '/contrast analysis/ellipses_data_with_weights_2sigma_' + str(exp) 

ellipse_data_2s = load_obj(ell_path + '.pkl')

if os.path.isfile(pred_path + '.pkl'):
    print ('Loading predictions')
    local_contrasts = load_obj(pred_path + '.pkl')
    mean_intensities = load_obj(rootF + '/contrast analysis/original_images_mean_intensities_2sigma_liu2022_exp' + str(exp) + '.pkl')
else:
    print ('Calculating predictions...')

    local_contrasts = np.empty((len(cells_clustering),Nimgs,2))
    mean_intensities = np.empty((len(cells_clustering),Nimgs,2))

    for index in range(len(cells_clustering)):
        cell_nb = cells_clustering[index]
        print (cell_nb)
        if cell_nb in selected_cells and cell_nb not in skip_cells:
            ### Get ellipse points
            if exp in [20221213, 20221214, 20230105]:
                cell_nb = str(cell_nb)
            ellipsepoints = ellipse_data_2s[cell_nb]['ellipse_points']
            weights = ellipse_data_2s[cell_nb]['weights']
            ### Local contrast for each image
            for iImg in range(Nimgs):
                for iPol in [0, 1]:
                    # load image
                    im = load_image_as_on_MEA(path_to_original_images + 'image_{}_image.png'.format(Imgs[iImg]) )
                    if iPol == 1:
                        im = abs(im - 1)
                    ellipse_weights = weights[ellipsepoints[:,1], ellipsepoints[:,0]]
                    mean_int, lsc = local_spatial_contrast(im, ellipsepoints, ellipse_weights)
                    
                    mean_intensities[index,iImg,iPol] = mean_int
                    local_contrasts[index,iImg,iPol] = lsc

Calculating predictions...
5
10
20
22
41
45
55
56
62
63
66
73
78
83
86
99
100
102
112
119
125
126
130
136
138
144
153
158
162
168
195
210
220
238
241
247
260
263
270
271
273
276
280
283
284
287
292
296
300
316
318
329
333
335
356
359
360
361
368
370
380
382
387
389
401
406
407
411
415
419
420
421
422
424
430
435
438
439
441
447
449
450
452
457
460
463
464
466
469
470
471
472
478
480
482
486
493
496
497
498
506
511
513
515
529
532
544
546
576
580
592
605
613


In [30]:
### Save predictions
save_path = rootF + '/contrast analysis/' 
                
save_obj(local_contrasts, save_path + '/original_images_local_contrast_2sigma_liu2022_exp' + str(exp))
save_obj(mean_intensities, save_path + '/original_images_mean_intensities_2sigma_liu2022_exp' + str(exp))